<a href="https://colab.research.google.com/github/SA6246/NYU_Tandon_Databases_Spring/blob/main/NYU_Tandon_Databases_Spring/project%20/project_final_ver3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies
!wget https://raw.githubusercontent.com/SA6246/NYU_Tandon_Databases_Spring/main/project/sources/createdb.sql createdb.sql > /dev/null 2>&1

In [ ]:
!wget https://raw.githubusercontent.com/SA6246/NYU_Tandon_Databases_Spring/main/project/sources/index.html index.html > /dev/null 2>&1

In [ ]:
# Install dependencies
!pip install mysql-connector-python > /dev/null 2>&1

In [ ]:
# Install dependencies
!apt-get -y install mysql-server > /dev/null 2>&1

In [ ]:
# Start SQL Server
!service mysql start > /dev/null 2>&1

In [ ]:
# Create our new Database and create a root account.
!mysql -u root -proot -e "ALTER USER 'root'@'localhost' IDENTIFIED BY 'root'; source createdb.sql;" > /dev/null 2>&1

In [ ]:
# Modify Database ROOT user access
!mysql -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH 'mysql_native_password' BY 'root';FLUSH PRIVILEGES;" > /dev/null 2>&1

In [ ]:
# Define the HTML template as a string
# Used a Generic Bootstrap Template

file_path = "/content/index.html"  # Update with the actual file name

with open(file_path, "r") as file:
    template = file.read()

In [ ]:
# Expose our notebook endpoint
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

In [ ]:
# @title
from flask import Flask, render_template_string, request
import pandas as pd
import mysql.connector

app = Flask(__name__)

@app.route("/")
def home():
    return render_template_string(template)

@app.route("/execute_query", methods=["POST"])
def execute_query():
    # Get the SQL query from the form data
    query = request.form.get("sqlQuery")

    # Pass out command to our query function
    table_html, operation_info = execute_sql_and_display(query)

    # Render the template with the results
    return render_template_string(template,
                                  output=table_html,
                                  operation_info=operation_info)

def execute_sql_and_display(command):

    # Create a connection to the MySQL server
    conn = mysql.connector.connect(user='root',
                                   password='root',
                                   host='localhost')

    # Create a cursor to interact with the MySQL server
    cursor = conn.cursor()

    # Execute the SQL command
    cursor.execute(command)

    # Check operation type
    operation_info = "No specific operation detected"
    if command.strip().lower().startswith("insert"):
        operation_info = "INSERT operation detected"
    elif command.strip().lower().startswith("update"):
        operation_info = "UPDATE operation detected"
    elif command.strip().lower().startswith("delete"):
        operation_info = "DELETE operation detected"
    else:
        operation_info = "READ operation detected"

    # If it's a non-SELECT operation, return operation_info only.
    if command.strip().lower().startswith(("insert", "update", "delete")):
        return None, operation_info

    # Fetch all the results
    records = cursor.fetchall()

    # Create a DataFrame from the records
    df = pd.DataFrame(records)

    # Get column names
    column_names = [col[0] for col in cursor.description]

    # Set column names to the DataFrame
    df.columns = column_names

    # Convert DataFrame to HTML table with column names
    table_html = df.to_html(index=False, header=True)

    return table_html, operation_info

if __name__ == "__main__":
    app.run()
